#Librairie 

In [0]:
!pip install pydantic openpyxl
!pip install pypdf
!pip install -qU langchain-community langchain-openai
!pip install pdf2image Pillow
!pip install pdfplumber
!pip install tqdm
!pip install unstructured

dbutils.library.restartPython()

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.r

In [0]:
!pip install openpyxl
import pandas as pd
import os
import json
from dataclasses import dataclass
import base64
import requests
from typing import Dict, List, Optional, Any, Union
from PIL import Image
import matplotlib.pyplot as plt
from openpyxl.utils import get_column_letter
import openpyxl
from matplotlib.table import Table
from pathlib import Path
from tqdm import tqdm


Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


#Init SecureGPT

In [0]:

@dataclass
class AuthConfig:
    """Configuration for authentication"""
    base_url: str
    client_id: str
    client_secret: str
    scope: str

class AuthClient:
    """Client for handling authentication"""
    def __init__(self, config: AuthConfig):
        self.config = config

    def get_token(self) -> Dict[str, str]:
        """Get OAuth2 access token"""
        try:
            response = requests.post(
                f"{self.config.base_url}/as/token.oauth2",
                headers={"Content-Type": "application/x-www-form-urlencoded"},
                data={
                    "grant_type": "client_credentials",
                    "client_id": self.config.client_id,
                    "client_secret": self.config.client_secret,
                    "scope": self.config.scope
                }
            )
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            raise Exception(f"Authentication failed: {str(e)}")


In [0]:
def get_auth_token() -> str:
    """Helper function to get authentication token"""
    config = AuthConfig(
        base_url="https://onelogin.stg.axa.com",
        client_id="VJKkswZb",
        client_secret="yG3pl6nkYhvJlQVD1y7xbZBl692YRytdpUJeODKg5BAutoEgxhkqzXLFYGqH5zlp",
        scope="urn:grp:chatgpt"
    )
    client = AuthClient(config)
    result = client.get_token()
    return result.get("access_token", "")

class SecureGPTChat:
    """Main class for interacting with SecureGPT API"""
    def __init__(self, 
                token: str,
                temperature: float = 0.05,
                base_url: str = "https://api-pp.se.axa-go.axa.com/ago-m365-securegpt-bapi-v1-vrs",
                model: str = "gpt-4o-2024-08-06"):
        self.token = token
        self.temperature = temperature
        self.base_url = base_url
        self.model = model
        self.costs = {
            'input_token': 2.50/1_000_000,
            'output_token': 10.00/1_000_000,
            'base_image_cost': 0.001913
        }

    def calculate_cost(self, response: Dict, image_path: Optional[str] = None) -> Dict[str, float]:
        """Calculate cost of API usage"""
        total_cost = 0
        metrics = {'token_cost': 0, 'image_cost': 0}

        if 'usage' in response:
            input_tokens = response['usage'].get('prompt_tokens', 0)
            output_tokens = response['usage'].get('completion_tokens', 0)
            metrics['token_cost'] = (input_tokens * self.costs['input_token'] + 
                                   output_tokens * self.costs['output_token'])
            total_cost += metrics['token_cost']

        if image_path:
            try:
                with Image.open(image_path) as img:
                    width, height = img.size
                    pixels = width * height
                    metrics['image_cost'] = (pixels / (1024 * 1024)) * self.costs['base_image_cost']
                    total_cost += metrics['image_cost']
            except Exception as e:
                print(f"Error calculating image cost: {str(e)}")

        metrics['total_cost'] = total_cost
        return metrics

    def encode_image(self, image_path: str) -> str:
        """Encode image to base64"""
        if image_path.startswith('data:image'):
            return image_path.split('base64,')[1]
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    def prepare_messages(self, messages: List[Dict[str, Any]], 
                        image_path: Optional[str] = None) -> List[Dict]:
        """Prepare messages for API request"""
        formatted_messages = []
        for msg in messages:
            content = msg["content"]
            if image_path and msg == messages[-1]:
                content = [
                    {
                        "type": "image_url",
                        "image_url": f"data:image/jpeg;base64,{self.encode_image(image_path)}",
                        "detail": "auto"
                    },
                    {"type": "text", "text": content}
                ]
            formatted_messages.append({
                "role": msg.get("role", "user"),
                "content": content
            })
        return formatted_messages

    def chat(self, messages: List[Dict[str, Any]], 
             image_path: Optional[str] = None, 
             **kwargs) -> Dict:
        """Send chat request to API"""
        url = f"{self.base_url}/deployments/{self.model}/chat/completions"
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {self.token}"
        }
        data = {
            "messages": self.prepare_messages(messages, image_path),
            "temperature": self.temperature,
            "max_tokens": kwargs.get("max_tokens", 12000)
        }

        try:
            response = requests.post(url, headers=headers, json=data)
            response.raise_for_status()
            api_response = response.json()
            cost_metrics = self.calculate_cost(api_response, image_path)
            api_response['cost_metrics'] = cost_metrics
            return api_response
        except requests.exceptions.RequestException as e:
            print(f"API Error Details: {response.text if 'response' in locals() else 'No response'}")
            raise Exception(f"API call failed: {str(e)}")

In [0]:
def create_chat_interface():
    """Create an interactive chat interface with the model"""
    token = get_auth_token()
    model = SecureGPTChat(
        temperature=0.0,
        model="gpt-4o-mini-2024-07-18",
        token=token
    )
    history = []
    
    def chat_with_model():
        while True:
            user_input = input("\nVous: ")
            if user_input.lower() in ['quit', 'exit', 'bye']:
                print("\nAu revoir!")
                break
                
            # Add message to history
            history.append({"role": "user", "content": user_input})
            
            # Get model response
            try:
                response = model.chat(history)
                bot_response = response['choices'][0]['message']['content']
                history.append({"role": "assistant", "content": bot_response})
                print("\nAssistant:", bot_response)
            except Exception as e:
                print(f"\nErreur: {str(e)}")
                continue
    
    return chat_with_model


com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:447)
	at com.databricks.spark.chauffeur.ChauffeurState.cancelExecution(ChauffeurState.scala:1315)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:1032)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:573)
	at com.databricks.logging.UsageLogging.executeThunkAndCaptureResultTags$1(UsageLogging.scala:669)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperationWithResultTags$4(UsageLogging.scala:687)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:426)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(AttributionContext.scala:216)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:424)
	at com.databricks.logging.Usa

#Transformation onglet excel (referentiel garantie) en json 

##onglet to image 

In [0]:
class ExcelStructureExtractor:
    """Class to handle Excel structure extraction and conversion to JSON"""
    def __init__(self, excel_file: str):
        self.excel_file = excel_file
        self.workbook = None
        
    def initialize_workbook(self):
        """Initialize the Excel workbook"""
        try:
            self.workbook = openpyxl.load_workbook(self.excel_file)
            return True
        except Exception as e:
            print(f"Error loading Excel file: {str(e)}")
            return False
    
    def get_sheet_names(self) -> List[str]:
        """Get all sheet names from the workbook"""
        if not self.workbook:
            self.initialize_workbook()
        return self.workbook.sheetnames
    
    def create_output_directories(self):
        """Create necessary output directories"""
        Path('temp_images').mkdir(exist_ok=True)
        Path('json_structures').mkdir(exist_ok=True)

    def excel_sheet_to_image(self, sheet_name: str, output_path: str):
        if not self.workbook:
            self.initialize_workbook()
            
        ws = self.workbook[sheet_name]
        
        # Prétraitement du texte avec wrapping
        data = []
        for row in ws.rows:
            processed_row = []
            for cell in row:
                text = str(cell.value) if cell.value is not None else ''
                # Wrap le texte tous les 30 caractères
                wrapped_text = '\n'.join([text[i:i+30] for i in range(0, len(text), 30)])
                processed_row.append(wrapped_text)
            data.append(processed_row)
        
        # Calcul des hauteurs avec plus d'espace pour le texte
        row_heights = []
        for row in data:
            max_lines = max(len(str(cell).split('\n')) for cell in row)
            row_heights.append(max_lines)
        
        # Calcul des largeurs avec contrainte
        col_widths = [min(max(len(line) for cell in col for line in str(cell).split('\n')), 30) 
                    for col in zip(*data)]
        
        # Dimensions ajustées
        total_width = min(sum(col_widths) * 0.12, 40)
        total_height = min(sum(row_heights) * 0.4, 30)  # Plus de hauteur pour le texte
        
        fig, ax = plt.subplots(figsize=(total_width, total_height))
        ax.axis('off')
        
        table = ax.table(
            cellText=data,
            cellLoc='left',
            loc='center',
            edges='BRTL'
        )
        
        table.auto_set_font_size(False)
        table.set_fontsize(8)  # Police réduite pour plus de texte
        
        # Ajustement des cellules avec plus d'espace vertical
        for i, row in enumerate(data):
            for j in range(len(row)):
                cell = table[i, j]
                cell.set_text_props(wrap=True, va='center')
                cell._text.set_text(data[i][j])
                
                # Plus de hauteur pour les cellules avec beaucoup de texte
                num_lines = len(str(data[i][j]).split('\n'))
                cell.set_height(max(num_lines * 0.03, 0.1))
                cell.set_width(min(col_widths[j] * 0.015, 0.1))
        
        plt.savefig(output_path, dpi=300, bbox_inches='tight', pad_inches=0.1)
        plt.close()


    def excel_sheet_to_image2(self, sheet_name: str, output_path: str):
        """
        Essaye de convertir un onglet Excel (feuille `sheet_name`) en image PNG,
        en dimensionnant (approx.) le tableau à la quantité de texte.
        """
        if not self.workbook:
            self.initialize_workbook()
        ws = self.workbook[sheet_name]

        # -- Récupérer le contenu cellulaire en mémoire --
        data = []
        for row in ws.rows:
            row_values = []
            for cell in row:
                txt = str(cell.value) if cell.value is not None else ""
                # On peut faire un léger wrapping manuel (tous les 30 caractères)
                # si l’on veut injecter des \n
                wrapped_txt = "\n".join(txt[i:i+30] for i in range(0, len(txt), 30))
                row_values.append(wrapped_txt)
            data.append(row_values)

        # Sécurité si la feuille est vide
        if not data:
            print(f"La feuille {sheet_name} est vide ou inexistante.")
            return

        # Nombre de colonnes/rows effectifs
        max_row = len(data)
        max_col = max(len(r) for r in data)

        # -- Calculer largeur / hauteur de chaque col/row en se basant sur la longueur de texte --
        # On va estimer qu'un caractère prend ~0.12 pouce (très approximatif)
        # et qu'une ligne de texte prend ~0.2 pouce de hauteur.
        width_per_char = 0.12
        height_per_line = 0.2

        # 1) Largeur de chaque colonne
        col_widths = []
        for col_idx in range(max_col):
            max_len_in_col = 0
            for row_idx in range(max_row):
                if col_idx < len(data[row_idx]):
                    cell_lines = data[row_idx][col_idx].split('\n')
                    # on regarde la ligne la plus longue
                    longest_line = max(cell_lines, key=len) if cell_lines else ""
                    max_len_in_col = max(max_len_in_col, len(longest_line))
            # On se garde une marge +1
            col_widths.append((max_len_in_col + 1) * width_per_char)

        # 2) Hauteur de chaque ligne
        row_heights = []
        for row_idx in range(max_row):
            # On regarde, dans chaque colonne, combien de lignes “\n” on a
            # et on prend le max
            max_nb_lines = 1
            for col_idx in range(len(data[row_idx])):
                nb_lines = data[row_idx][col_idx].count('\n') + 1
                max_nb_lines = max(max_nb_lines, nb_lines)
            row_heights.append(max_nb_lines * height_per_line)

        # On obtient une taille totale du tableau
        total_width = sum(col_widths)
        total_height = sum(row_heights)

        # On ajoute un peu de marge autour, par ex. 1 pouce
        fig_margin = 1.0
        fig, ax = plt.subplots(figsize=(total_width + fig_margin, total_height + fig_margin))
        ax.set_axis_off()

        # -- Construction du tableau matplotlib --
        table = Table(ax, bbox=[0, 0, 1, 1])
        current_y = 0.0

        for row_idx in range(max_row):
            current_x = 0.0
            for col_idx in range(max_col):
                cell_txt = ""
                if col_idx < len(data[row_idx]):
                    cell_txt = data[row_idx][col_idx]

                cw = col_widths[col_idx]
                ch = row_heights[row_idx]

                # Ajout d'une cellule
                cell = table.add_cell(row=row_idx,
                                    col=col_idx,
                                    width=cw/(total_width),
                                    height=ch/(total_height),
                                    text=cell_txt,
                                    loc='center',
                                    edgecolor='black')

                # On active le wrap
                cell.set_text_props(wrap=True, va='center')
            # Fin col
        table.auto_set_font_size(False)
        table.set_fontsize(8)

        table.rows = max_row
        table.cols = max_col
        ax.add_table(table)

        # -- Sauvegarde de la figure --
        plt.savefig(output_path, dpi=200, bbox_inches='tight')
        plt.close()
        print(f"Image enregistrée dans : {output_path}")

##Image ton Json 

In [0]:
def extract_structure_with_securegpt(model: SecureGPTChat, 
                                  image_path: str, 
                                  example_json: Dict) -> Dict:
   messages = [
       {
           "role": "system",
           "content": f"""Tu es un expert en assurance analysant des tableaux Excel de paramètres d'endoso.

Structure requise:
0. S'il y a pas de colonne enfin si tu vois qu'il y a comme colonne standard parameters ce n'est pas une colonne mais c'est juste pour dire que les variables à côté et en dessous sont de type "std" donc quand il n'y a pas de colonne tu mets juste column_parameters : '' en gros un vide. S'il y a quelque chose qui ressemble à une garantie là tu mets sinon rien et surtout tu ne mets pas standard parameters...
1. Les paramètres sous "Standard Parameters" doivent avoir type="std"
1bis. Les paramètres sous "Parameters in exception" doivent avoir type="exception"
1ter. Idem pour les options il peut y avoir d'autres cas que tu verras forcément sur l'image en dessous de standard, exception etc..
2. Les paramètres doivent être regroupés par "column_parameters" si des colonnes sont visibles
3. Si pas de colonnes visibles, utiliser uniquement "raw_parameters" à remplir et column_parameters : ""

Voici un exemple de la structure attendue:
{json.dumps(example_json, indent=2)}

Points importants:
- Créer les column_parameters uniquement si des colonnes sont visibles dans le tableau
- Standard/Exception définissent uniquement le type du paramètre, pas la structure, il n'apparaissent pas dans la structure
- Garder la même structure que l'exemple fourni
- Respecter les noms exacts des champs présents dans l'exemple
- Copier les valeurs textuelles exactes du tableau : TRES IMPORTANT C'EST A DIRE MEME S'IL Y A UN TEXTE LONG AVEC PLUSIEURS PHRASES DANS UNE CELLULE TU NE METS PAS QUE "TEXT" TU RECOPIES TOUT LE TEXTE.

Retourne uniquement le JSON."""
       },
       {
           "role": "user", 
           "content": "Extrais la structure JSON en respectant les règles prédéfinies."
       }
   ]
   
   response = model.chat(messages, image_path=image_path)
   return json.loads(response['choices'][0]['message']['content'].strip().replace('```json', '').replace('```', ''))


def extract_excel_to_text(excel_file, sheet_name):
   import pandas as pd
   df = pd.read_excel(excel_file, sheet_name=sheet_name)
   text_content = df.to_string(index=False)
   with open('paste.txt', 'w', encoding='utf-8') as f:
       f.write(text_content)
   return text_content

def transform_table_with_llm():
   # Charger l'exemple
   with open('output_referential_example.json', 'r') as f:
       example_json = json.load(f)
       
   # Lire le contenu 
   with open('paste.txt', 'r') as f:
       table_content = f.read()

   messages = [
       {
           "role": "system",
           "content": f"""Tu es un expert en transformation de tableaux Excel en JSON. Ta tâche est de transformer ce tableau en JSON en suivant EXACTEMENT la structure de l'exemple fourni.

Voici l'exemple de structure à suivre:
{json.dumps(example_json, indent=2)}

Points clés:
- Suit EXACTEMENT la même structure que l'exemple
- Les paramètres sous "Standard Parameters" ont type="std"
- Les paramètres sous "Parameters in exception" ont type="exception"
- Conserve TOUS les paramètres avec leurs value_type exacts
- Inclut editable:true, active:true pour tous les paramètres
- Extrait toutes les colonnes et sous-colonnes visibles
- Chaque colonne principale (Cesarea, Parto Normal) devient un column_parameter_name
- Les sous-sections (Pago directo, Reembolso) deviennent des column_parameter_details
- Les paramètres deviennent des row_parameters
- Tu mettras les valeurs textuelles exactes du tableau pour condition generale dans "description" par exemple

Retourne uniquement le JSON, sans texte explicatif."""
       },
       {
           "role": "user",
           "content": f"Transforme ce tableau en JSON:\n{table_content}"
       }
   ]

   model = SecureGPTChat(
       temperature=0,
       model="gpt-4o-2024-08-06", 
       token=get_auth_token()
   )

   response = model.chat(messages)
   return json.loads(response['choices'][0]['message']['content'].strip().replace('```json', '').replace('```', ''))

# Utilisation
text = extract_excel_to_text('input_referentials.xlsx', 'endoso_65')
json_output = transform_table_with_llm()

with open('output.json', 'w', encoding='utf-8') as f:
   json.dump(json_output, f, indent=2, ensure_ascii=False)

In [0]:
def process_excel_structure(excel_file: str, model: SecureGPTChat):
    special_endosos = ['toto']  # Liste des endosos à traiter différemment
    
    extractor = ExcelStructureExtractor(excel_file)
    extractor.create_output_directories()
    sheet_names = extractor.get_sheet_names()
    
    with open('output_referential_example.json', 'r') as f:
        example_json = json.load(f)
    
    for sheet_name in tqdm(sheet_names, desc="Processing sheets"):
        if 'endoso' not in sheet_name.lower():
            continue
            
        print(f"\nProcessing sheet: {sheet_name}")
        try:
            if sheet_name not in special_endosos:
                # Méthode alternative pour les endosos spéciaux
                text = extract_excel_to_text(excel_file, sheet_name)
                structure = transform_table_with_llm()
            else:
                # Méthode standard
                image_path = f"temp_images/{sheet_name}.png"
                extractor.excel_sheet_to_image(sheet_name, image_path)
                structure = extract_structure_with_securegpt(model, image_path, example_json)
            
            # Sauvegarde
            json_path = f"json_structures/{sheet_name}.json"
            with open(json_path, 'w', encoding='utf-8') as f:
                json.dump(structure, f, indent=2, ensure_ascii=False)
                
        except Exception as e:
            print(f"Error processing sheet {sheet_name}: {str(e)}")
            continue

#Brouillon test 

In [0]:
model = SecureGPTChat(
   temperature=0.0,
   model="gpt-4o-2024-08-06",
   token=get_auth_token()
)
numero = "122"
extractor = ExcelStructureExtractor('input_referentials.xlsx')
extractor.excel_sheet_to_image2(f"endoso_{numero}", f"temp_images/endoso_{numero}.png")


with open('output_referential_example.json', 'r') as f:
            example_json = json.load(f)
image_path = f"temp_images/endoso_{numero}.png"
model = SecureGPTChat(
   temperature=0.0,
   model="gpt-4o-2024-08-06",
   token=get_auth_token()
)
extract_structure_with_securegpt(model,image_path,example_json)